In [9]:
import pandas as pd
import os

In [ ]:
import os
import pandas as pd

# Directories and file paths
playersShiftDirectory = "../MLBDATA/Raw/ShiftData"
seasonsFilePath       = "../MLBDATA/Raw/BatterData/BattingData2015-2024NoCovid.csv"
archetypesFilePath    = "../MLBDATA/Processed/BatterData/PlayerArchetypesClusters2016-2022NoCovid.csv"
firstYear             = 2015
lastPreBanYear        = 2022
firstPostBanYear      = 2023
outputFile            = "../MLBDATA/Processed/BatterData/ModelingBattingData.csv"

# Load season‐level data and player‐level archetypes
dfSeasons     = pd.read_csv(seasonsFilePath)
dfArchetypes  = pd.read_csv(archetypesFilePath)

# 1) Load & clean initial shift file (2016)
dfPlayerShift = pd.read_csv(os.path.join(playersShiftDirectory, "POSPlayer2016.csv"))
dfPlayerShift.drop(columns=["team_id","total_pa"], inplace=True)
dfPlayerShift.rename(columns={"name":"last_name, first_name"}, inplace=True)

# 2) Append all other years of shift data
for fn in os.listdir(playersShiftDirectory):
    if fn.startswith("POSPlayer") and fn.endswith(".csv") and fn!="POSPlayer2016.csv":
        tmp = pd.read_csv(os.path.join(playersShiftDirectory, fn))
        tmp.drop(columns=["team_id","total_pa"], inplace=True)
        tmp.rename(columns={"name":"last_name, first_name"}, inplace=True)
        dfPlayerShift = pd.concat([dfPlayerShift, tmp], ignore_index=True)

# 3) Keep only pre‐ban years
dfPlayerShift = dfPlayerShift[dfPlayerShift["year"] <= lastPreBanYear]

# 4) Rename shift percentage to shift_rate and drop everything else except that
dfPlayerShift.rename(columns={"pa_shift_percent":"shift_rate"}, inplace=True)

# Now drop the raw shift counts & other woba‐by‐shift stats, but KEEP shift_rate
colsToDrop = [
    "year",
    "pa_shifts",      "woba_shift",
    "pa_non_shifts",  "pa_non_shift_percent",  "woba_non_shift",
    "pa_shades",      "pa_shade_percent",      "woba_shade",
    "pa_non_shades",  "pa_non_shade_percent",  "woba_non_shade"
]
dfPlayerShift.drop(columns=colsToDrop, inplace=True)

# Deduplicate on name (so each player has one shift_rate)
dfPlayerShift.drop_duplicates(subset=["last_name, first_name"], inplace=True)

# 5) Merge archetype clusters into the seasons data
dfModel = (
    dfSeasons
      .merge(
          dfArchetypes[["player_id","cluster","Archetype"]],
          on="player_id",
          how="inner"
      )
)

# 6) Normalize the name key in both for merging
def normalize_name(s):
    return s.str.replace(r"[,\s]+","",regex=True).str.lower()

dfModel["last_name, first_name"]    = normalize_name(dfModel["last_name, first_name"])
dfPlayerShift["last_name, first_name"] = normalize_name(dfPlayerShift["last_name, first_name"])

# 7) Merge in the shift_rate
dfModel = dfModel.merge(
    dfPlayerShift,
    on="last_name, first_name",
    how="inner"
)

# 8) Flag “extremely shifted”—here, shift_rate ≥ 50
dfModel["extreme_shift"] = (dfModel["shift_rate"] >= 50).astype(int)

# 9) Restrict to players present pre‐ and post‐ban
preIds    = set(dfModel[dfModel["year"] <= lastPreBanYear]["player_id"])
postIds   = set(dfModel[dfModel["year"] >= firstPostBanYear]["player_id"])
commonIds = preIds.intersection(postIds)
dfClean   = dfModel[dfModel["player_id"].isin(commonIds)].reset_index(drop=True)

# 10) Export for modeling
dfClean.to_csv(outputFile, index=False)

# 11) Quick summary
print(f"Filtered to {len(commonIds)} players and {len(dfClean)} rows.")
print("Unique players per cluster:")
print(dfClean.groupby("cluster")["player_id"].nunique().sort_index())
print("\nShift‐rate summary by cluster:")
print(dfClean.groupby("cluster")["shift_rate"].agg(["mean","median","max"]))
print("\nCount of extremely shifted players by cluster:")
print(dfClean.groupby("cluster")["extreme_shift"].sum())
print("\nCount of non-extreme shifted players by cluster:")
print(
    dfClean[dfClean['extreme_shift']==0]
      .groupby('cluster')['player_id']
      .nunique()
      .sort_index()
)


Filtered to 268 players and 1390 rows.
Unique players per cluster:
cluster
0     71
1    104
2     93
Name: player_id, dtype: int64

Shift‐rate summary by cluster:
              mean  median   max
cluster                         
0         8.924852     1.9  60.4
1        18.601440     8.7  93.2
2        22.183138    12.3  86.0

Count of extremely shifted players by cluster:
cluster
0     5
1    97
2    79
Name: extreme_shift, dtype: int64

Count of non-extreme shifted players by cluster:
cluster
0    70
1    89
2    72
Name: player_id, dtype: int64


: 